# Harmonic Coordinates on a Regular Grid

This is where I test harmonic coordinates on a regular grid to verify its behavior.

In [ ]:
import numpy as np
import igl
import meshplot as mp
import scipy.sparse as sp

In [ ]:
nx, ny = 5, 5
v, f = igl.triangulated_grid(nx, ny)
v = np.hstack([v, np.zeros((v.shape[0], 1))]) # add z-axis

In [ ]:
mp.plot(v, f, shading={"wireframe": True}) # display regular grid

In [ ]:
cage_indices = [0, 2, 4, 10, 14, 20, 22, 24]

p = mp.plot(v, f, shading={"wireframe": True})
p.add_points(v[cage_indices], shading={"point_size": 0.1, "point_color":"red"})

In [ ]:
def compute_hi(V, F, cage_indices, ci):
    
    L = igl.cotmatrix(V, F)
    b = np.zeros(V.shape[0])
    
    L = L.tolil()
    
    for i in cage_indices:
        L[i, :] = 0
        L[i, i] = 1
        b[i] = 1 if i == ci else 0
    
    L = L.tocsc()
    hi = sp.linalg.spsolve(L, b)

    return hi

In [ ]:
def compute_H(V, F, cage_indices):
    H = np.zeros((V.shape[0], len(cage_indices)))

    for i, ci in enumerate(cage_indices):
        hi = compute_hi(V, F, cage_indices, ci)
        H[:, i] = hi

    return H

In [ ]:
H = compute_H(v, f, cage_indices)
row_sums = np.sum(H, axis=1)
print(f"Min sum: {row_sums.min()}, Max sum: {row_sums.max()}") # the each row's sum should be (close to) 1.

In [ ]:
# display weights
ci = 1
p = mp.plot(v, f, c=H[:,ci], shading={"wireframe": True})
p.add_points(v[[cage_indices[ci]]], shading={"point_size": 0.15, "point_color":"red"})